In [1]:
import tensorflow as tf
print(tf.__version__)

2.0.0-rc1


In [22]:
K.clear_session()

In [23]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [24]:
import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.layers import Conv2D, Convolution2D, Layer, UpSampling2D, Conv2DTranspose, Flatten, Reshape, Lambda, BatchNormalization,Dense, Activation
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.losses import mse, binary_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend


import os
import numpy as np
import math
from PIL import Image

In [25]:
image_size = 128
inputShape = (128, 128, 3)
latentSize = 64

#encoder_type deepmind_enc
data_dir = '/home/dovi/home/dataset'
graph_dir = '/content/drive/My Drive/Colab Notebooks/VAE/graphs'
test_image_folder = '/content/drive/My Drive/Colab Notebooks/VAE/test_images/celeb'
encoder_type = 'deepmind_enc'
save_dir = '/content/drive/My Drive/Colab Notebooks/VAE/saved_models'
save_dir_img = '/content/drive/My Drive/Colab Notebooks/VAE/new_vae_images'
val_split = 0.2

train_batch_size = 64
val_batch_size = 64 
optimizer = 'ADAM' 
base_learning_rate = 0.01
num_epochs = 100 
scheduler_epoch = 5 
decay_factor = 0.01 
capacity = 25
multi_process=True


depth = 7
nb_dense_block = 1
growth_rate = 12
nb_filter = 16
dropout_rate=0.2
weight_decay=1E-4
latentConstraints='bvae', 
beta=10.
training=None

In [26]:
def post_process_output(generated_image):
    generated_image[generated_image > 0.5] = 0.5
    generated_image[generated_image < -0.5] = -0.5
    gen_image = np.uint8((generated_image + 0.5) * 255)
    return gen_image


def batch_gen(generator_object):
    while True:
        data = generator_object.next()
        yield [pre_process_input(data[0])], [pre_process_input(data[0])]
      
      
def pre_process_input(image_array):
    x = np.asarray(image_array)
    y = x - 0.5
    return y

In [27]:
class SampleLayer(Layer):
    def __init__(self, gamma, capacity, slname, **kwargs):
        super(SampleLayer, self).__init__(**kwargs)
        self.gamma = gamma
        self.max_capacity = capacity
        self.slname = slname

    def build(self, input_shape):
        super(SampleLayer, self).build(input_shape)
        self.built = True

    def call(self, layer_inputs, **kwargs):
        if len(layer_inputs) != 2:
            raise Exception('input layers must be a list: mean and stddev')
        if len(K.int_shape(layer_inputs[0])) != 2 or len(K.int_shape(layer_inputs[1])) != 2:
            raise Exception('input shape is not a vector [batchSize, latentSize]')

        mean = layer_inputs[0]
        log_var = layer_inputs[1]

        batch = K.shape(mean)[0]
        dim = K.int_shape(mean)[1]

        latent_loss = -0.5 * (1 + log_var - K.square(mean) - K.exp(log_var))
        latent_loss = K.sum(latent_loss, axis=1, keepdims=True)
        latent_loss = K.mean(latent_loss)
        latent_loss = self.gamma * K.abs(latent_loss - self.max_capacity)

        latent_loss = K.reshape(latent_loss, [1, 1])

        epsilon = K.random_normal(shape=(batch, dim), mean=0., stddev=1.)
        layer_output = mean + K.exp(0.5 * log_var) * epsilon

        self.add_loss(losses=[latent_loss], inputs=[layer_inputs])

        return layer_output

    def compute_output_shape(self, input_shape):
        return input_shape[0]

    def get_config(self):
        config = {
            'gamma': self.gamma,
            'capacity': self.max_capacity,
            'name': self.slname
        }
        base_config = super(SampleLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

class Architecture(object):
    '''
    generic architecture template
    '''
    def __init__(self, inputShape=None, batchSize=None, latentSize=None):
        '''
        params:
        ---------
        inputShape : tuple
            the shape of the input, expecting 3-dim images (h, w, 3)
        batchSize : int
            the number of samples in a batch
        latentSize : int
            the number of dimensions in the two output distribution vectors -
            mean and std-deviation
        latentSize : Bool or None
            True forces resampling, False forces no resampling, None chooses based on K.learning_phase()
        '''
        self.inputShape = inputShape
        self.batchSize = batchSize
        self.latentSize = latentSize

        self.model = self.Build()

    def Build(self):
        raise NotImplementedError('architecture must implement Build function')


class Encoder(Architecture):

    def __init__(self, input_shape=128, latent_dim=10):
        self._input_shape = input_shape
        self._z_dim = latent_dim

    @staticmethod
    def conv_func(input_layer):
        x = Conv2D(filters=32, kernel_size=4, strides=2, padding='same')(input_layer)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        return x

    def build(self, vae_gamma, vae_capacity):
        init = x = Input((self._input_shape, self._input_shape, 3))

        for _ in range(5):
            x = self.conv_func(x)

        x = Flatten()(x)

        x = Dense(units=256)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Dense(units=256)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Dense(units=self._z_dim * 2, activation='linear')(x)
        # Beta-VAE implementation as follows

        z_mean = Dense(units=self._z_dim, activation='linear', name='sample_mean')(x)
        z_log_var = Dense(units=self._z_dim, activation='linear', name='sample_log_var')(x)
        embed_layer = SampleLayer(gamma=vae_gamma, capacity=vae_capacity, slname='sampling_layer')([z_mean, 
                                                                                                    z_log_var])

        return init, embed_layer

    def create_encoder(self, vae_gamma, vae_capacity):
        init, embed_layer = self.build(vae_gamma=vae_gamma, vae_capacity=vae_capacity)
        model = Model(inputs=[init], outputs=[embed_layer])
        return init, model


class Decoder(Architecture):
    def __init__(self, decoder_input, latent_dim, output_shape):
        self._dec_input = decoder_input
        self._latent_dim = latent_dim

    @staticmethod
    def deconv_func(input_layer, pad_mode='same'):
        x = Conv2DTranspose(32, 4, strides=2, padding=pad_mode)(input_layer)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        return x

    def build(self, encoder_output):
        x = Dense(units=256)(encoder_output)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Dense(units=256)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Dense(units=512, activation='relu')(x)
        x = BatchNormalization()(x)
        x = Reshape((-1, 1, 512), input_shape=(512,))(x)

        for i in range(5):
            if i == 0:
                x = self.deconv_func(x, pad_mode='valid')
            else:
                x = self.deconv_func(x)
        x = Conv2DTranspose(3, 4, strides=2, padding='same', activation='tanh')(x)
        decoded = x
        return decoded

    def create_decoder(self):
        input_dec = Lambda(lambda x: x, name='decoder_inp')(self._dec_input)

        return self.build(encoder_output=input_dec)


In [28]:
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, Callback
import time

lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=decay_factor, mode='min', patience=scheduler_epoch, min_lr=1e-010)
timestamp = time.strftime("%d%m%Y", time.localtime())

save_file = os.path.join(save_dir, str(encoder_type + '_vae' + timestamp + '.hdf5'))
checkpoint = ModelCheckpoint(filepath=save_file, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)

In [29]:
def post_process_output(generated_image):
    generated_image[generated_image > 0.5] = 0.5
    generated_image[generated_image < -0.5] = -0.5
    gen_image = np.uint8((generated_image + 0.5) * 255)
    return gen_image


def batch_gen(generator_object):
    while True:
        data = generator_object.next()
        yield [pre_process_input(data[0])], [pre_process_input(data[0])]
      
      
def pre_process_input(image_array):
    x = np.asarray(image_array)
    y = x - 0.5
    return y

In [30]:
data_gen = ImageDataGenerator(rescale=1 / 255.,validation_split=val_split)

train_generator = data_gen.flow_from_directory(data_dir, target_size=(image_size, image_size), batch_size=train_batch_size, class_mode='categorical', subset='training')

validation_generator = data_gen.flow_from_directory(data_dir, target_size=(image_size, image_size), batch_size=val_batch_size, class_mode='categorical', subset='validation')

training_steps = math.ceil(train_generator.n / train_batch_size)
validation_steps = math.ceil(validation_generator.n / val_batch_size)

print('Steps per epoch in training: ', training_steps)
print('Steps per epoch in validation: ', validation_steps)

training_generator = batch_gen(generator_object=train_generator)

validation_generator = batch_gen(generator_object=validation_generator)

Found 191197 images belonging to 1 classes.
Found 47799 images belonging to 1 classes.
Steps per epoch in training:  2988
Steps per epoch in validation:  747


In [31]:
# This is how you build the autoencoder
enc = Encoder(input_shape=image_size, latent_dim=latentSize)
enc_input, enc_model = enc.create_encoder(vae_gamma=10, vae_capacity=0)
z = enc_model.output
dec = Decoder(decoder_input=z, latent_dim=latentSize, output_shape=image_size)
dec_output = dec.create_decoder()

model = Model(inputs=[enc_input], outputs=[dec_output])

print(model.summary())

model.compile(optimizer=optimizer, loss=['mean_squared_error'])

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 64, 64, 32)   1568        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 64, 64, 32)   128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 64, 64, 32)   0           batch_normalization[0][0]        
____________________________________________________________________________________________

In [ ]:
model.fit_generator(training_generator, steps_per_epoch=training_steps, epochs=num_epochs,
                validation_data=validation_generator,validation_steps=validation_steps)

Epoch 1/100
2988/2988 [==============================] - 1216s 407ms/step - loss: 1.0248 - val_loss: 0.0255
Epoch 2/100
2988/2988 [==============================] - 1626s 544ms/step - loss: 0.0303 - val_loss: 0.0199
Epoch 3/100
2988/2988 [==============================] - 1606s 537ms/step - loss: 0.0234 - val_loss: 0.0232
Epoch 4/100
2988/2988 [==============================] - 1689s 565ms/step - loss: 0.0187 - val_loss: 0.0459
Epoch 5/100
2988/2988 [==============================] - 1337s 448ms/step - loss: 0.0164 - val_loss: 0.0166
Epoch 6/100
2988/2988 [==============================] - 1235s 413ms/step - loss: 0.0279 - val_loss: 0.0160
Epoch 7/100
2988/2988 [==============================] - 1220s 408ms/step - loss: 0.0159 - val_loss: 0.0162
Epoch 8/100
 228/2988 [=>............................] - ETA: 14:57 - loss: 0.0159

In [ ]:
model.save_weights('deepMV')

In [ ]:
model.save_weights('deepMV.h5')

In [34]:
def my_vae_loss(y_true, y_pred):
    xent_loss = image_size * image_size * mse(K.flatten(inputs), K.flatten(outputs))
    kl_loss = - 0.5 * beta * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    vae_loss = K.mean(xent_loss + kl_loss)
    return vae_loss

In [35]:
vae.compile(optimizer='rmsprop', loss=my_vae_loss)

In [36]:
vae.summary()

Model: "vae"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   [(None, 128, 128, 3)]     0         
_________________________________________________________________
encoder (Model)              [(None, 64), (None, 64),  316448    
_________________________________________________________________
decoder (Model)              (None, None, 128, 3)      548131    
Total params: 864,579
Trainable params: 860,867
Non-trainable params: 3,712
_________________________________________________________________


In [37]:
vae.fit_generator(training_generator, steps_per_epoch=training_steps, epochs=num_epochs,validation_data=validation_generator,validation_steps=validation_steps)

Epoch 1/100


ValueError: No gradients provided for any variable: ['conv2d/kernel:0', 'conv2d/bias:0', 'batch_normalization/gamma:0', 'batch_normalization/beta:0', 'conv2d_1/kernel:0', 'conv2d_1/bias:0', 'batch_normalization_1/gamma:0', 'batch_normalization_1/beta:0', 'conv2d_2/kernel:0', 'conv2d_2/bias:0', 'batch_normalization_2/gamma:0', 'batch_normalization_2/beta:0', 'conv2d_3/kernel:0', 'conv2d_3/bias:0', 'batch_normalization_3/gamma:0', 'batch_normalization_3/beta:0', 'conv2d_4/kernel:0', 'conv2d_4/bias:0', 'batch_normalization_4/gamma:0', 'batch_normalization_4/beta:0', 'dense/kernel:0', 'dense/bias:0', 'batch_normalization_5/gamma:0', 'batch_normalization_5/beta:0', 'dense_1/kernel:0', 'dense_1/bias:0', 'batch_normalization_6/gamma:0', 'batch_normalization_6/beta:0', 'dense_2/kernel:0', 'dense_2/bias:0', 'sample_mean/kernel:0', 'sample_mean/bias:0', 'sample_log_var/kernel:0', 'sample_log_var/bias:0', 'dense_3/kernel:0', 'dense_3/bias:0', 'batch_normalization_7/gamma:0', 'batch_normalization_

In [24]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

InternalError: cudaGetDevice() failed. Status: CUDA driver version is insufficient for CUDA runtime version

In [31]:
print(tf.keras.__version__)

2.2.4-tf


In [55]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = Session(config = config)
tf.compat.v1.disable_eager_execution()

AttributeError: module 'tensorflow' has no attribute 'ConfigProto'